In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image

Using TensorFlow backend.


In [7]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import Dropout
K.set_image_data_format('channels_last')

In [2]:
df=pd.read_csv("C:/Users/jaina/Downloads/ESC-50-master/meta/esc50.csv")

In [3]:
processed_dir="C:/Users/jaina/Downloads/ESC-50-master/pro_dir_images/"

In [4]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for file in os.listdir(img_folder):    
            image_path= os.path.join(img_folder, file)
            img = image.load_img(image_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = preprocess_input(x)
            img_data_array.append(x)
            cat=df[df["filename"].str[:-4]==file[:-4]].target.values[0]
            class_name.append(cat)
    return img_data_array, class_name

In [5]:
# extract the image array and class name
img_data, class_name =create_dataset(processed_dir)

In [6]:
X=np.array(img_data, np.float32)
y=np.array(class_name)

In [8]:
model=keras.models.Sequential()
VGG = VGG16(include_top=False, input_shape=(224,224,3),pooling='avg')
model.add(VGG)
VGG.summary()
for layer in model.layers:
    layer.trainable=False
model.add(Dropout(rate=0.5))
model.add(Dense(216,activation='relu'))
model.add(Dense(50,activation='softmax'))

for layer in model.layers:
    print(layer.name, layer.trainable)

model.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____________________________________________

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [11]:
from keras import optimizers
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train,y_train, epochs=100,batch_size=None,validation_data=(X_test,y_test))
model.summary()


Train on 1600 samples, validate on 400 samples
Epoch 1/100
1600/1600 [==============================] - 26s 16ms/step - loss: 5.1931 - accuracy: 0.0675 - val_loss: 3.0892 - val_accuracy: 0.1975
Epoch 2/100
1600/1600 [==============================] - 19s 12ms/step - loss: 3.2595 - accuracy: 0.1981 - val_loss: 2.5231 - val_accuracy: 0.3675
Epoch 3/100
1600/1600 [==============================] - 19s 12ms/step - loss: 2.6803 - accuracy: 0.2931 - val_loss: 2.1340 - val_accuracy: 0.4300
Epoch 4/100
1600/1600 [==============================] - 19s 12ms/step - loss: 2.3477 - accuracy: 0.3500 - val_loss: 1.9328 - val_accuracy: 0.4825
Epoch 5/100
1600/1600 [==============================] - 19s 12ms/step - loss: 2.0323 - accuracy: 0.4437 - val_loss: 1.7135 - val_accuracy: 0.5375
Epoch 6/100
1600/1600 [==============================] - 19s 12ms/step - loss: 1.8225 - accuracy: 0.4831 - val_loss: 1.6262 - val_accuracy: 0.5825
Epoch 7/100
1600/1600 [==============================] - 19s 12ms/step